Bibliografia:

Para o modelo de submissão:
https://www.kaggle.com/code/philculliton/nlp-getting-started-tutorial/notebook

Conceitos de ML:
https://tatianaesc.medium.com/machine-learning-conceitos-e-modelos-f0373bf4f445

Visualização:
https://realpython.com/twitter-sentiment-python-docker-elasticsearch-kibana/

Conceito VADER:
https://ojs.aaai.org/index.php/ICWSM/article/view/14550/14399

algoritmo de classificação utilizado no titanic
wording bading / baggle of words/ df-tf (atribui pesos as palvras)
feater extraction -> transformar o texto em uma coluna de banco de dados

1)Limepeza das palavras: tirar as stop words, lematização, braking bad(?)

## 1) Vizualização dos dados

In [2]:
import numpy as np
import pandas as pd
import nltk
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize, word_tokenize

In [3]:
from nltk.stem import WordNetLemmatizer

In [4]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [5]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
tweets = train_df['text']

## 2) Minimização dos ruídos

### Substituindo os espaços e lemmatizando as palavras chaves

In [7]:
filtro = train_df['keyword'].notnull()

In [8]:
sample_df_nonull=train_df[filtro]

In [9]:
len(sample_df_nonull)

7552

In [10]:
keyword = sample_df_nonull['keyword']

In [11]:
keywords = [x.replace('%20', ' ') for x in keyword]

In [12]:
lemmatizer = WordNetLemmatizer()

In [13]:
Keywords_words = [word_tokenize(x) for x in keywords]

In [14]:
key= []
for keywords in Keywords_words:
    words_lemmatized = []
    for word in keywords:
        words_lemmatized.append(lemmatizer.lemmatize(word))
    key.append(words_lemmatized)    

In [15]:
sample_df_nonull['keyword'] = key

C:\Users\yfrom\AppData\Local\Temp\ipykernel_3000\2651402788.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df_nonull['keyword'] = key


### Text

In [123]:
filtro = train_df['text'].notnull()
sample_text = train_df['text']

In [162]:
sample_text

0       our deeds are the reason of this earthquake ma...
1                   forest fire near la ronge sask canada
2       all residents asked to 'shelter in place' are ...
3        people receive wildfires evacuation orders in...
4       just got sent this photo from ruby alaska as s...
                              ...                        
7608    two giant cranes holding a bridge collapse int...
7609      the out of control wild fires in california ...
7610                        m  utckm s of volcano hawaii 
7611    police investigating after an e-bike collided ...
7612    the latest more homes razed by northern califo...
Name: text, Length: 7613, dtype: object

In [126]:
#Retirada dos links
sample_text = sample_text.str.replace('(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+/(?:[-\w.]|(?:%[\da-fA-F]{2}))+','')

C:\Users\yfrom\AppData\Local\Temp\ipykernel_3000\1460596230.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  sample_text = sample_text.str.replace('(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+/(?:[-\w.]|(?:%[\da-fA-F]{2}))+','')


In [128]:
#Retirada dos @
sample_text = sample_text.str.replace(r'@[a-zA-Z0-9_!.]{0,25}','')

C:\Users\yfrom\AppData\Local\Temp\ipykernel_3000\128505014.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  sample_text = sample_text.str.replace(r'@[a-zA-Z0-9_!.]{0,25}','')


In [161]:
#retirada de símbolos não alfanuméticos 
sample_text = sample_text.str.replace(r"[#,.;:?!()\[\]]","")

C:\Users\yfrom\AppData\Local\Temp\ipykernel_3000\2838354984.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  sample_text = sample_text.str.replace(r"[#,.;:?!()\[\]]","")


In [62]:
#retirada de \\x89Û_ \x89ÛÒ \x89Û \x89Û÷ (Não sei o que significa..)
sample_text = sample_text.replace("\\x89Û_",'')
sample_text = sample_text.replace('\x89ÛÓ','')
sample_text = sample_text.replace('\x89ÛÓ','')
sample_text = sample_text.replace('\x89ÛÒ','')
sample_text = sample_text.replace('\x89Û','')
sample_text = sample_text.replace('\x89Û÷','')

In [133]:
sample_text = sample_text.str.replace(r'[0-9]','')

C:\Users\yfrom\AppData\Local\Temp\ipykernel_3000\1194353961.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  sample_text = sample_text.str.replace(r'[0-9]','')


In [137]:
sample_text = sample_text.str.lower()